# 3.2 OpenAI API Deep Dive — Function Calling & Structured Outputs

## Playground Notebook

In this notebook, we'll explore two powerful features that make LLMs **actionable**:

| Feature | What It Does |
|---------|-------------|
| **Function Calling** | Model decides *when* to call a function and *what arguments* to pass |
| **Structured Outputs** | Guarantees the response matches an exact JSON schema |

The model **never executes functions** — it tells your code what to run, then uses the result.

> **Model:** `gpt-4o-mini` — supports both function calling and structured outputs.

---

In [1]:
import os
import json
import time
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown, HTML

load_dotenv()

MODEL = "gpt-4o-mini"
client = OpenAI()

print(f"\u2705 Client ready | Model: {MODEL}")

✅ Client ready | Model: gpt-4o-mini


In [2]:
# ============================================================
#  HELPER FUNCTIONS
# ============================================================

def chat(messages, max_tokens=150, **kwargs):
    """Send messages to OpenAI and display the response."""
    start = time.time()
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=max_tokens,
        **kwargs
    )
    elapsed = time.time() - start
    content = response.choices[0].message.content
    if content:
        display(Markdown(content))
    print(f"\n\u23f1\ufe0f {elapsed:.2f}s | Tokens: {response.usage.prompt_tokens}+{response.usage.completion_tokens}={response.usage.total_tokens}")
    return response


def show_messages(messages):
    """Pretty-print the message list being sent (handles both dicts and OpenAI objects)."""
    colors = {"system": "#e74c3c", "user": "#3498db", "assistant": "#2ecc71", "tool": "#f39c12"}
    html = ""
    for msg in messages:
        # Handle both dict and OpenAI message objects
        if isinstance(msg, dict):
            role = msg.get("role", "unknown")
            content = msg.get("content", "")
        else:
            role = getattr(msg, "role", "unknown")
            content = getattr(msg, "content", None)

        # Handle tool_calls (assistant messages with no content)
        if not content:
            tool_calls = msg.get("tool_calls", None) if isinstance(msg, dict) else getattr(msg, "tool_calls", None)
            if tool_calls:
                content = ", ".join(f"{tc.function.name}({tc.function.arguments})" for tc in tool_calls)
                content = f"[tool_calls] {content}"
            else:
                content = "(empty)"

        if len(str(content)) > 200:
            content = str(content)[:200] + "..."

        color = colors.get(role, "#888")
        html += (
            f'<div style="margin:6px 0;padding:8px 12px;border-left:4px solid {color};'
            f'background:#1e1e1e;border-radius:4px;">'
            f'<strong style="color:{color};text-transform:uppercase;">{role}</strong>'
            f'<br><span style="color:#ccc;">{content}</span></div>'
        )
    display(HTML(html))


print("\u2705 Helpers loaded")

✅ Helpers loaded


---

## 1. Function Calling — The Full Flow

```
1. You define functions and describe them to the model (tool schema)
2. User sends a message
3. Model responds with a tool_call (which function + what arguments)
4. Your code executes the actual function
5. You send the result back to the model
6. Model uses the result to generate the final response
```

### Experiment 1A: Define a Function + Tool Schema

In [3]:
# Step 1: Define the ACTUAL function your code will execute
def get_weather(location: str, unit: str = "celsius") -> dict:
    """Simulate a weather API call."""
    # In production, this would call a real weather API
    weather_data = {
        "new york": {"temp": 22, "condition": "Sunny"},
        "london":   {"temp": 15, "condition": "Cloudy"},
        "tokyo":    {"temp": 28, "condition": "Humid"},
    }
    data = weather_data.get(location.lower(), {"temp": 20, "condition": "Unknown"})
    if unit == "fahrenheit":
        data["temp"] = round(data["temp"] * 9/5 + 32)
    data["unit"] = unit
    data["location"] = location
    return data


# Step 2: Describe it to the model (tool schema)
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the current weather for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City name, e.g. 'New York'"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit"
                }
            },
            "required": ["location"]
        }
    }
}

print("\u2705 Function defined + tool schema ready")
print("\nTool schema sent to the model:")
print(json.dumps(weather_tool, indent=2))

✅ Function defined + tool schema ready

Tool schema sent to the model:
{
  "type": "function",
  "function": {
    "name": "get_weather",
    "description": "Get the current weather for a given location.",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "City name, e.g. 'New York'"
        },
        "unit": {
          "type": "string",
          "enum": [
            "celsius",
            "fahrenheit"
          ],
          "description": "Temperature unit"
        }
      },
      "required": [
        "location"
      ]
    }
  }
}


### Experiment 1B: Send Request — Model Decides to Call the Function

In [4]:
# Step 3: Send the request
messages = [
    {"role": "user", "content": "What's the weather in Tokyo?"}
]

show_messages(messages)
response = chat(messages, max_tokens=100, tools=[weather_tool])

# Step 4: Check if the model wants to call a function
choice = response.choices[0]
print(f"finish_reason: {choice.finish_reason}")

if choice.finish_reason == "tool_calls":
    tool_call = choice.message.tool_calls[0]
    print(f"\n\u2705 Model wants to call a function!")
    print(f"   Function:  {tool_call.function.name}")
    print(f"   Arguments: {tool_call.function.arguments}")
    print(f"   Call ID:   {tool_call.id}")
else:
    print(f"Model responded directly (no tool call needed)")


⏱️ 1.23s | Tokens: 77+14=91
finish_reason: tool_calls

✅ Model wants to call a function!
   Function:  get_weather
   Arguments: {"location":"Tokyo"}
   Call ID:   call_pnoQkJqrRw0v7CbmcZ6zIVvp


### Experiment 1C: Execute the Function & Send Result Back

In [5]:
# Step 5: Execute the function with the model's arguments
args = json.loads(tool_call.function.arguments)
result = get_weather(**args)
print(f"Function result: {result}")

# Step 6: Send the result back to the model
messages.append(choice.message)  # include the assistant's tool_call message
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": json.dumps(result)
})

# Model generates final response using the function result
print(f"\n{'=' * 60}")
print("Sending tool result back to model for final response:")
print(f"{'=' * 60}")
show_messages(messages)
final_response = chat(messages, max_tokens=80)

Function result: {'temp': 28, 'condition': 'Humid', 'unit': 'celsius', 'location': 'Tokyo'}

Sending tool result back to model for final response:


The current weather in Tokyo is 28°C and humid.


⏱️ 0.72s | Tokens: 61+12=73


### Experiment 1D: `tool_choice` — Controlling When Functions Are Called

| Value | Behavior |
|-------|----------|
| `"auto"` | Model decides (default) |
| `"none"` | Model will NOT call any function |
| `"required"` | Model MUST call a function |
| `{"type": "function", "function": {"name": "..."}}` | Force a specific function |

In [6]:
# tool_choice="none" — model answers directly, ignores tools
print("=" * 60)
print("tool_choice='none' — Model answers directly, ignores tools")
print("=" * 60)

messages = [{"role": "user", "content": "What's the weather in London?"}]
show_messages(messages)
response = chat(messages, max_tokens=60, tools=[weather_tool], tool_choice="none")
print(f"finish_reason: {response.choices[0].finish_reason}")

# tool_choice="required" — model MUST call a function
print(f"\n{'=' * 60}")
print("tool_choice='required' — Model forced to call function even for a joke")
print("=" * 60)

messages2 = [{"role": "user", "content": "Tell me a joke."}]
show_messages(messages2)
response2 = chat(messages2, max_tokens=60, tools=[weather_tool], tool_choice="required")

tc = response2.choices[0].message.tool_calls[0]
print(f"  Function: {tc.function.name}")
print(f"  Args: {tc.function.arguments}")

tool_choice='none' — Model answers directly, ignores tools


What temperature unit would you like for the weather in London: Celsius or Fahrenheit?


⏱️ 0.87s | Tokens: 78+16=94
finish_reason: stop

tool_choice='required' — Model forced to call function even for a joke



⏱️ 0.90s | Tokens: 76+15=91
  Function: get_weather
  Args: {"location":"New York"}


### Experiment 1E: Multiple Tools — Model Picks the Right One

In [7]:
# Define a second function
def calculate(expression: str) -> str:
    """Evaluate a math expression safely."""
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {e}"


calc_tool = {
    "type": "function",
    "function": {
        "name": "calculate",
        "description": "Evaluate a mathematical expression and return the result.",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Math expression, e.g. '2 * 3 + 4'"
                }
            },
            "required": ["expression"]
        }
    }
}

tools = [weather_tool, calc_tool]

# Test with different queries — model picks the right tool
queries = [
    "What's the weather in New York?",
    "What is 15 * 23 + 7?",
    "Hello, how are you?"  # no tool needed
]

for q in queries:
    print(f"\n{'=' * 50}")
    messages = [{"role": "user", "content": q}]
    show_messages(messages)
    resp = chat(messages, max_tokens=60, tools=tools)

    choice = resp.choices[0]
    if choice.finish_reason == "tool_calls":
        for tc in choice.message.tool_calls:
            print(f"  \u2192 Calls: {tc.function.name}({tc.function.arguments})")
    else:
        print(f"  \u2192 Direct response (no tool needed)")


⏱️ 0.99s | Tokens: 116+15=131
  → Calls: get_weather({"location":"New York"})




⏱️ 1.00s | Tokens: 120+19=139
  → Calls: calculate({"expression":"15 * 23 + 7"})



Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?


⏱️ 1.47s | Tokens: 115+31=146
  → Direct response (no tool needed)


### Experiment 1F: Parallel Tool Calls

When a user asks something that requires **multiple functions**, the model can call them all at once.

In [8]:
# Ask about weather in two cities — model may call get_weather twice in parallel
messages = [{"role": "user", "content": "Compare weather in Tokyo and London."}]
show_messages(messages)
response = chat(messages, max_tokens=100, tools=[weather_tool])

choice = response.choices[0]
if choice.finish_reason == "tool_calls":
    print(f"\u2705 Model made {len(choice.message.tool_calls)} parallel tool calls!\n")

    # Execute all functions
    messages.append(choice.message)

    available_functions = {"get_weather": get_weather, "calculate": calculate}

    for tc in choice.message.tool_calls:
        fn = available_functions[tc.function.name]
        args = json.loads(tc.function.arguments)
        result = fn(**args)
        print(f"  {tc.function.name}({args}) \u2192 {result}")
        messages.append({
            "role": "tool",
            "tool_call_id": tc.id,
            "content": json.dumps(result)
        })

    # Get final response
    print(f"\n{'=' * 60}")
    print("Final response with both results:")
    print(f"{'=' * 60}")
    show_messages(messages)
    final = chat(messages, max_tokens=100)


⏱️ 2.11s | Tokens: 78+54=132
✅ Model made 2 parallel tool calls!

  get_weather({'location': 'Tokyo', 'unit': 'celsius'}) → {'temp': 28, 'condition': 'Humid', 'unit': 'celsius', 'location': 'Tokyo'}
  get_weather({'location': 'London', 'unit': 'celsius'}) → {'temp': 15, 'condition': 'Cloudy', 'unit': 'celsius', 'location': 'London'}

Final response with both results:


As of the latest weather updates:

- **Tokyo**: The temperature is 28°C with humid conditions.
- **London**: The temperature is 15°C and it is cloudy.

In summary, Tokyo is currently much warmer and more humid compared to the cooler and cloudier weather in London.


⏱️ 1.32s | Tokens: 132+60=192


---

## 2. Structured Outputs — Guaranteed JSON Schema

Structured Outputs guarantee the model's response always matches a schema you define — valid JSON, every field present, correct data types.

### Method 1: JSON Mode (Simple, Less Strict)

Guarantees valid JSON but does **not** enforce a specific schema.

In [9]:
messages = [
    {"role": "system", "content": "Extract info as JSON with keys: name, topic, key_points (list)."},
    {"role": "user", "content": "Marie Curie discovered radioactivity, won two Nobel Prizes, and pioneered radiation therapy."}
]

show_messages(messages)
response = chat(messages, max_tokens=120, response_format={"type": "json_object"})

text = response.choices[0].message.content
try:
    parsed = json.loads(text)
    print(f"\u2705 Valid JSON! Keys: {list(parsed.keys())}")
except json.JSONDecodeError:
    print("\u274c Not valid JSON")

{
  "name": "Marie Curie",
  "topic": "Achievements in Science",
  "key_points": [
    "Discovered radioactivity",
    "Won two Nobel Prizes",
    "Pioneered radiation therapy"
  ]
}


⏱️ 2.02s | Tokens: 45+51=96
✅ Valid JSON! Keys: ['name', 'topic', 'key_points']


### Method 2: Structured Outputs with Pydantic (Strict, Recommended)

Define your exact schema using **Pydantic** — OpenAI guarantees the response matches it every time.

In [10]:
from pydantic import BaseModel


class PersonInfo(BaseModel):
    name: str
    topic: str
    key_points: list[str]


messages = [
    {"role": "system", "content": "Extract structured information from the text."},
    {"role": "user", "content": "Alan Turing invented the concept of the Turing machine, broke the Enigma code, and laid the foundations of computer science."}
]

show_messages(messages)

response = client.beta.chat.completions.parse(
    model=MODEL,
    messages=messages,
    response_format=PersonInfo,
    max_tokens=120
)

result = response.choices[0].message.parsed
print(f"\u2705 Parsed into Pydantic object:")
print(f"   Name:       {result.name}")
print(f"   Topic:      {result.topic}")
print(f"   Key Points: {result.key_points}")
print(f"   Type:       {type(result).__name__}")
print(f"\n\u23f1\ufe0f Tokens: {response.usage.prompt_tokens}+{response.usage.completion_tokens}={response.usage.total_tokens}")

✅ Parsed into Pydantic object:
   Name:       Alan Turing
   Topic:      Contributions to Computer Science
   Key Points: ['Invented the concept of the Turing machine', 'Broke the Enigma code', 'Laid the foundations of computer science']
   Type:       PersonInfo

⏱️ Tokens: 110+44=154


### Experiment 2C: Nested Schemas — Complex Structured Data

In [11]:
from typing import Optional


class Address(BaseModel):
    city: str
    country: str


class Company(BaseModel):
    name: str
    industry: str
    founded_year: Optional[int]
    headquarters: Address
    products: list[str]


messages = [
    {"role": "system", "content": "Extract company information."},
    {"role": "user", "content": "Tesla, founded in 2003, is an electric vehicle company headquartered in Austin, USA. They make the Model 3, Model Y, and Cybertruck."}
]

show_messages(messages)

response = client.beta.chat.completions.parse(
    model=MODEL,
    messages=messages,
    response_format=Company,
    max_tokens=120
)

company = response.choices[0].message.parsed
print(f"\u2705 Parsed nested Pydantic object:")
print(f"   Company:   {company.name}")
print(f"   Industry:  {company.industry}")
print(f"   Founded:   {company.founded_year}")
print(f"   HQ:        {company.headquarters.city}, {company.headquarters.country}")
print(f"   Products:  {company.products}")
print(f"\n\u23f1\ufe0f Tokens: {response.usage.prompt_tokens}+{response.usage.completion_tokens}={response.usage.total_tokens}")

✅ Parsed nested Pydantic object:
   Company:   Tesla
   Industry:  Electric Vehicles
   Founded:   2003
   HQ:        Austin, USA
   Products:  ['Model 3', 'Model Y', 'Cybertruck']

⏱️ Tokens: 183+42=225


### JSON Mode vs Structured Outputs — When to Use Which

| Feature | JSON Mode | Structured Outputs (Pydantic) |
|---------|-----------|-------------------------------|
| Valid JSON? | \u2705 Yes | \u2705 Yes |
| Schema enforced? | \u274c No | \u2705 Yes — every field, every type |
| Setup effort | Minimal | Define Pydantic models |
| Parsing needed? | Yes (json.loads) | No — direct `.parsed` access |
| Best for | Quick extraction | Production data pipelines |

---

## 3. Combining Function Calling + Structured Outputs

In [12]:
# A complete tool-calling loop with structured final output

class WeatherReport(BaseModel):
    location: str
    temperature: int
    condition: str
    recommendation: str


# Step 1: Ask with tools
messages = [
    {"role": "system", "content": "You are a weather assistant. Use the weather tool, then summarize."},
    {"role": "user", "content": "What's the weather in New York?"}
]

print("=" * 60)
print("Step 1: Send request with tools")
print("=" * 60)
show_messages(messages)
resp = chat(messages, max_tokens=100, tools=[weather_tool])

# Step 2: Execute tool call
if resp.choices[0].finish_reason == "tool_calls":
    tc = resp.choices[0].message.tool_calls[0]
    args = json.loads(tc.function.arguments)
    result = get_weather(**args)

    print(f"\n{'=' * 60}")
    print(f"Step 2: Execute function \u2192 {result}")
    print(f"{'=' * 60}")

    messages.append(resp.choices[0].message)
    messages.append({"role": "tool", "tool_call_id": tc.id, "content": json.dumps(result)})

    # Step 3: Get structured final response
    print(f"\n{'=' * 60}")
    print("Step 3: Get structured final response")
    print(f"{'=' * 60}")
    show_messages(messages)

    final = client.beta.chat.completions.parse(
        model=MODEL, messages=messages,
        response_format=WeatherReport, max_tokens=100
    )

    report = final.choices[0].message.parsed
    print(f"\u2705 Structured Report:")
    print(f"   Location:       {report.location}")
    print(f"   Temperature:    {report.temperature}")
    print(f"   Condition:      {report.condition}")
    print(f"   Recommendation: {report.recommendation}")
    print(f"\n\u23f1\ufe0f Tokens: {final.usage.prompt_tokens}+{final.usage.completion_tokens}={final.usage.total_tokens}")

Step 1: Send request with tools



⏱️ 0.73s | Tokens: 92+15=107

Step 2: Execute function → {'temp': 22, 'condition': 'Sunny', 'unit': 'celsius', 'location': 'New York'}

Step 3: Get structured final response


✅ Structured Report:
   Location:       New York
   Temperature:    22
   Condition:      Sunny
   Recommendation: A great day for outdoor activities!

⏱️ Tokens: 150+25=175


---

## 4. Sandbox — Try It Yourself!

In [13]:
# ============================================================
#  SANDBOX — Define your own function + tool schema
# ============================================================

# Example: A simple lookup tool
def lookup_capital(country: str) -> str:
    capitals = {"france": "Paris", "japan": "Tokyo", "brazil": "Brasilia", "india": "New Delhi"}
    return capitals.get(country.lower(), "Unknown")


capital_tool = {
    "type": "function",
    "function": {
        "name": "lookup_capital",
        "description": "Look up the capital city of a country.",
        "parameters": {
            "type": "object",
            "properties": {
                "country": {"type": "string", "description": "Country name"}
            },
            "required": ["country"]
        }
    }
}

# Try it!
messages = [{"role": "user", "content": "What is the capital of India?"}]
show_messages(messages)
resp = chat(messages, max_tokens=60, tools=[capital_tool])

choice = resp.choices[0]
if choice.finish_reason == "tool_calls":
    tc = choice.message.tool_calls[0]
    args = json.loads(tc.function.arguments)
    result = lookup_capital(**args)
    print(f"Tool called: {tc.function.name}({args}) \u2192 {result}")

    messages.append(choice.message)
    messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})

    print(f"\n{'=' * 60}")
    print("Final response:")
    print(f"{'=' * 60}")
    show_messages(messages)
    final = chat(messages, max_tokens=60)


⏱️ 0.81s | Tokens: 58+15=73
Tool called: lookup_capital({'country': 'India'}) → New Delhi

Final response:


The capital of India is New Delhi.


⏱️ 0.74s | Tokens: 40+8=48


---

## Key Takeaways

| Concept | What to Remember |
|---------|------------------|
| **Function Calling** | Model decides *when* and *what* to call — your code executes it |
| **Tool Schema** | JSON description of your function — name, description, parameters |
| **tool_calls** | The model's response when it wants to call a function |
| **tool_choice** | `auto` (default), `none`, `required`, or force a specific function |
| **Parallel Calls** | Model can call multiple functions in one response |
| **JSON Mode** | `response_format={"type": "json_object"}` — valid JSON, no schema |
| **Structured Outputs** | `response_format=PydanticModel` — exact schema, typed, guaranteed |
| **`.parsed`** | Direct access to Pydantic object — no manual JSON parsing |